In [15]:
import pyomo.environ as pyo
import pandas as pd
from pathlib import Path
import preprocess_data as ppd

## Data

In [16]:
data_path = Path("../data/")
output_path = Path("../output/")
df, adjacent_matrix = ppd.get_df_adj(data_path, 2021)
model = pyo.AbstractModel()

## Defs

In [17]:
def param_adjacent(m, i, j):
    return int(j in adjacent_matrix[i])


def param_pop(m, i):
    return df["population"][i - 1]


def con_a(m, i):
    return sum((m.a[i, j] * m.x[j]) for j in m.J) >= m.y[i]


def con_x(m):
    return sum(m.x[j] for j in m.J) <= m.k


def obj_sum(m):
    return pyo.summation(m.p, m.y)

## Constraints

In [18]:
# value of n (number of counties)
model.n = 88

# TOSET limit on number of pirincipal places of buisnesses opened (init to 5)
model.k = 15

# range of i and j (iterating over counties)
model.I = pyo.RangeSet(1, model.n)
model.J = pyo.RangeSet(1, model.n)

model.p = pyo.Param(model.I, initialize=param_pop)  # population of county i

# model.a = pyo.Set(model.I, model.J, within=pyo.Binary, initialize=param_adjacent)  # 1 if county i and j are adjacent
model.a = pyo.Param(
    model.I, model.J, domain=pyo.Binary, initialize=param_adjacent
)  # 1 if county i and j are adjacent

model.x = pyo.Var(
    model.J, domain=pyo.Binary
)  # 1 if principal place of business is opened in county j
model.y = pyo.Var(model.I, domain=pyo.Binary)  # 1 if county i is covered

model.obj = pyo.Objective(rule=obj_sum, sense=pyo.minimize)

model.a_constraint = pyo.Constraint(model.I, rule=con_a)
model.x_constraint = pyo.Constraint(rule=con_x)

## Solve

In [19]:
instance = model.create_instance()
# opt = pyo.SolverFactory("ipopt", executable="/home/adb/anaconda3/bin/ipopt")
opt = pyo.SolverFactory("glpk", executable="/home/adb/anaconda3/bin/glpsol")
# opt = pyo.SolverFactory("glpk", executable="/usr/local/Caskroom/miniconda/base/bin/glpsol")
results = opt.solve(instance).write()
results
# instance.pprint()
# instance.solutions.store_to(results)
# model21.pprint()
# model21.x.get_values()
# solution.print_data()
# solution.solutions.store_to(results)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 0.0
  Upper bound: 0.0
  Number of objectives: 1
  Number of constraints: 90
  Number of variables: 177
  Number of nonzeros: 727
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.003222227096557617
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------

## Verify

In [20]:
sol_dict = instance.x.get_values()
sol1 = [k for k, v in sol_dict.items() if v == 1]
sol1_camm = ppd.county_ids_to_camm_ids(df, sol1)
pd.DataFrame(
    zip(
        sol1,
        sol1_camm,
    ),
    columns=[
        "obtained_sol1",
        "obtained_sol1_camm",
    ],
)

,obtained_sol1,obtained_sol1_camm


In [21]:
sol_save = pd.Series(sol1)
sol_save.to_csv(
    output_path / "solution_model_2_population.csv", index=False, header=["county_id"]
)

/tmp/ipykernel_15138/3864745534.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sol_save = pd.Series(sol1)
